In [1]:
from transformers import AutoModel
import torch


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("./glm_models", trust_remote_code=True, device_map='auto')
# model = AutoModel.from_pretrained("./glm_models", load_in_8bit=True, trust_remote_code=True, device_map="auto")

/root/miniconda3/envs/gl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.40s/it]


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./glm_models", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [3]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "output/adapter_model.bin"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


/root/miniconda3/envs/gl/lib/python3.8/site-packages/peft/tuners/lora.py:191: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [4]:
import json

instructions = json.load(open("data/alpaca-zh.json"))

In [8]:
answers = []
from cover_alpaca2jsonl import format_example


with torch.no_grad():
    for idx, item in enumerate(instructions[0:5]):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

Instruction: 给出三个保持健康的秘诀。
Answer: 1. 均衡饮食:吃健康的食物,包括水果、蔬菜、全谷物和蛋白质。
2. 锻炼:每天锻炼有助于保持身体健康和心理健康。
3. 睡眠:充足的睡眠有助于保持身体和心理健康。
### 1.Answer:
 1.均衡饮食，多吃水果和蔬菜。
2. 定期锻炼，保持身体活跃和强壮。
3.充足的睡眠，保持规律的作息时间。 


Instruction: 什么是三原色?
Answer: 三原色是红、蓝和黄。
### 2.Answer:
 三原色是红、蓝、黄。 


Instruction: 描述原子的结构。
Answer: 原子由带正电荷的电子围绕带负电荷的质子旋转组成。电子带负电,质子带正电。电子和质子之间存在电子云,每个电子云都有特定的电荷和位置。
### 3.Answer:
 原子由原子核组成，原子核包含质子和中子，周围环绕着围绕原子核运行的电子。质子和中子带正电荷，而电子带负电荷，从而形成整体中性原子。每个粒子的数量决定了原子序数和原子类型。 


Instruction: 我们怎样才能减少空气污染?
Answer: 减少空气污染的关键是减少燃烧化石燃料产生的二氧化碳和其他污染物。这可以通过使用可再生能源、减少汽车排放以及鼓励使用公共交通和骑自行车等方式来减少。此外,鼓励人们使用电动汽车、鼓励使用太阳能和风能等可再生能源以及鼓励使用电动汽车和骑自行车。此外,政府应鼓励人们使用公共交通工具,并应鼓励人们使用电动汽车。
### 4.Answer:
 减少空气污染的方法有很多，例如转向可再生能源、鼓励使用公共交通工具、禁止燃烧化石燃料、实施工业源减排政策以及实施机动车排放标准。此外，个人可以通过减少汽车使用、避免燃烧木材等材料以及改用节能电器来为减少空气污染做出自己的贡献。 


Instruction: 描述一次你不得不做出艰难决定的经历。
Answer: 我最近不得不做出一个艰难的决定。我有两个选项,一个是继续我的学业,另一个是放弃我的学业并从事更具创造性的工作。我选择了放弃我的学业,因为我相信我可以通过从事更具创造性的工作来充分利用我的技能和知识。虽然这是一个艰难的决定,但我认为它是正确的,因为我可以追求我真正想要的事情。
### 5.Answer:
 当我在一家建筑公司担任项目经理时，我不得不做出一个艰难的决定